## SQL Queries Of MIMIC IV Database In Order To Create One Streamlined Table For EDA, and ML model Building

Introduction
This notebook demonstrates an end-to-end SQL-driven data cleaning and transformation workflow applied to a real-world dataset. The project simulates a professional data wrangling task using PostgreSQL, where raw, messy tabular data is systematically normalized, standardized, and cleaned through a series of structured queries and procedural logic.

Key aspects include:

Exploratory Data Analysis (EDA) using SELECT, COUNT, and GROUP BY to identify inconsistencies and nulls

Column-level standardization, including formatting dates, fixing case sensitivity, and splitting multi-value fields

Null value handling through conditional logic and CASE statements

Joins and CTEs (Common Table Expressions) to restructure tables and enrich the dataset

Final output generation via materialized views or export-ready tables

In [7]:
# Template for a DATAFRAME.
# This function is used to provide basic Dataframe analysis to Explore data that can due utilized for exploring different tables;
# Help us better understand the different tables it's values and it usability for our study on Spinal Chord porject.

import pandas as pd

def explore_dataframe(df):
    """
    Explores and summarizes the given DataFrame.
    
    Parameters:
        df (DataFrame): The DataFrame to analyze.
        
    Returns:
        None
    """
    print("--------------- DataFrame Exploration Summary ---------------")

    # Basic Information and Records
    print("\nNumber of records (rows):", df.shape[0])  # Total number of rows
    print("Number of columns:", df.shape[1])  # Total number of columns
    print("\nColumn Names:")
    print(df.columns)  # List of column names

    # Preview the Data
    print("\nFirst 5 rows of the DataFrame:")
    print(df.head())  # Display the first 5 rows
    print("\nLast 5 rows of the DataFrame:")
    print(df.tail())  # Display the last 5 rows

    # Data Types and Structure
    print("\nInformation about DataFrame (columns and data types):")
    print(df.info())  # Information about columns, types, non-null values

    # Summary Statistics
    print("\nSummary statistics for numerical columns:")
    print(df.describe())  # Summary stats: count, mean, std, min, max, etc.

    # Check Missing Data
    print("\nNumber of missing (NaN) values per column:")
    print(df.isnull().sum())  # Count NaN values in each column

    # Count Non-Null Values in Each Column
    print("\nNumber of non-null (non-NaN) values per column:")
    print(df.count())  # Count of non-null values for each column

    # Unique Values in a Specific Column
    column_name = 'column_name_here'  # Replace with your column name
    if column_name in df.columns:
        print(f"\nUnique values in '{column_name}':")
        print(df[column_name].unique())  # Display unique values

    # Grouping Data
    # Example: Replace 'column_name' with the column to group by
    if 'column_name' in df.columns:
        grouped = df.groupby('column_name').size()  # Group and count
        print("\nGrouped Data (count per category):")
        print(grouped)

    # Filter Example
    # Example: Replace 'column_name' and 'specific_value' with your criteria
    if 'column_name' in df.columns:
        filtered_df = df[df['column_name'] == 'specific_value']
        print("\nFiltered DataFrame (example):")
        print(filtered_df)

    print("\n--------------- End of Exploration Summary ---------------")

# Example: Call this function on a DataFrame
# Assume `df` is your DataFrame
# explore_dataframe(df)

In [9]:
# Code to test SQL Lite for the first time to be able to write Standard SQL to review csv/compressed files. 
import pandas as pd
import sqlite3

# Read the .csv.gz file into a Pandas DataFrame
df = pd.read_csv("c:/stefan/icu/icustays.csv.gz")

# Connect to (or create) a database file
conn = sqlite3.connect("d:/mimic.db")

# Write the DataFrame to a SQL table
df.to_sql("icustays", conn, if_exists="replace", index=False)

# Run a SQL query

query = "SELECT * FROM icustays limit 5;"
result_df = pd.read_sql_query(query, conn)

# Display the query result
print(result_df)

conn.close()


   subject_id   hadm_id   stay_id  \
0    10000032  29079034  39553978   
1    10000980  26913865  39765666   
2    10001217  24597018  37067082   
3    10001217  27703517  34592300   
4    10001725  25563031  31205490   

                                     first_careunit  \
0                Medical Intensive Care Unit (MICU)   
1                Medical Intensive Care Unit (MICU)   
2               Surgical Intensive Care Unit (SICU)   
3               Surgical Intensive Care Unit (SICU)   
4  Medical/Surgical Intensive Care Unit (MICU/SICU)   

                                      last_careunit               intime  \
0                Medical Intensive Care Unit (MICU)  2180-07-23 14:00:00   
1                Medical Intensive Care Unit (MICU)  2189-06-27 08:42:00   
2               Surgical Intensive Care Unit (SICU)  2157-11-20 19:18:02   
3               Surgical Intensive Care Unit (SICU)  2157-12-19 15:42:24   
4  Medical/Surgical Intensive Care Unit (MICU/SICU)  2110-04-11 15

In [23]:
# In this example we are reviewing table loaded in chunks as the table is very largea us sql lite and chunks at a time.
import pandas as pd
import sqlite3

# Read the .csv.gz file in chunks using Pandas
chunksize = 10000
conn = sqlite3.connect("c:/Stefan/mimic.db")

for chunk in pd.read_csv("c:/stefan/icu/chartevents.csv.gz", compression='gzip', chunksize=chunksize):
    chunk.to_sql("chartevents", conn, if_exists="append", index=False)

# Query the database
query = "SELECT * FROM chartevents LIMIT 5;"
result_df = pd.read_sql_query(query, conn)
print(result_df)

conn.close()


   subject_id   hadm_id   stay_id  caregiver_id            charttime  \
0    10000032  29079034  39553978       47007.0  2180-07-23 21:01:00   
1    10000032  29079034  39553978       47007.0  2180-07-23 21:01:00   
2    10000032  29079034  39553978       47007.0  2180-07-23 21:01:00   
3    10000032  29079034  39553978       47007.0  2180-07-23 22:00:00   
4    10000032  29079034  39553978       47007.0  2180-07-23 22:00:00   

             storetime  itemid value  valuenum valueuom  warning  
0  2180-07-23 22:15:00  220179    82      82.0     mmHg      0.0  
1  2180-07-23 22:15:00  220180    59      59.0     mmHg      0.0  
2  2180-07-23 22:15:00  220181    63      63.0     mmHg      0.0  
3  2180-07-23 22:15:00  220045    94      94.0      bpm      0.0  
4  2180-07-23 22:15:00  220179    85      85.0     mmHg      0.0  


In [2]:
# To show what exsits in the mimic.db (sqllite), all the table names.
# Please note that tables below in SQL Lite database is result table are all CAPITAL letters. 
# Simply put the mixed cases tables are all input tables where all CAP tables are result tables.
import pandas as pd
import sqlite3

# Connect to (or create) a database file
conn = sqlite3.connect("e:/mimic.db")

# Query to list all tables in the database
query = "SELECT name FROM sqlite_master WHERE type='table';"

# Commit the changes (optional but good practice)
conn.commit()

# Execute the query and display the results
tables_df = pd.read_sql_query(query, conn)
print(tables_df)

# Close the connection
conn.close()


                                 name
0                         chartevents
1                    Ingredientevents
2                         inputevents
3                        outputevents
4                     procedureevents
5                      datetimeevents
6                             d_items
7                            icustays
8                       diagnoses_icd
9                     d_icd_diagnoses
10                   d_icd_procedures
11                     procedures_icd
12                            patient
13                         admissions
14                          labevents
15                 microbiologyevents
16                    spinal_icd_list
17            ICU_SPINAL_PATIENT_5290
18        ICU_SPINAL_PATIENT_FEATURES
19                      caregiver.csv
20                          caregiver
21  ICU_SPINAL_PATIENT_FEATURES_CHART


In [8]:
# Run everytime; Loads tables in sqllite database. Created a function. At the bottom are many exmaples of the tables loaded,
# using the load_csv_to_sqlite function in chunks incase some of the table are very large.
import pandas as pd
import sqlite3

def load_csv_to_sqlite(file_path, db_path, table_name, chunksize=10000):
    """
    Loads a compressed .csv.gz file into a SQLite database table.

    Parameters:
    - file_path (str): Path to the .csv.gz file to be loaded.
    - db_path (str): Path to the SQLite database file.
    - table_name (str): Name of the table to write data to.
    - chunksize (int): Number of rows to process per chunk (default is 10000).
    """
    try:
        # Connect to (or create) the SQLite database
        conn = sqlite3.connect(db_path)

        # Read the .csv.gz file in chunks and write to the database
        for chunk in pd.read_csv(file_path, compression='gzip', chunksize=chunksize):
            chunk.to_sql(table_name, conn, if_exists="append", index=False)

        # Query the database to confirm data has been written
        query = f"SELECT * FROM {table_name} LIMIT 5;"
        result_df = pd.read_sql_query(query, conn)
        print(result_df)

        # Close the connection
        conn.close()
        print(f"Data from {file_path} has been successfully loaded into the {table_name} table!")
    except Exception as e:
        print(f"An error occurred: {e}")


"""
load_csv_to_sqlite(
    file_path="c:/stefan/icu/Ingredientevents.csv.gz",
    db_path="e:/mimic.db",
    table_name="Ingredientevents"
)

load_csv_to_sqlite(
    file_path="c:/stefan/icu/inputevents.csv.gz",
    db_path="e:/mimic.db",
    table_name="inputevents"
)

load_csv_to_sqlite(
    file_path="c:/stefan/icu/outputevents.csv.gz",
    db_path="e:/mimic.db",
    table_name="outputevents"
)

load_csv_to_sqlite(
    file_path="c:/stefan/icu/procedureevents.csv.gz",
    db_path="e:/mimic.db",
    table_name="procedureevents"
)

load_csv_to_sqlite(
    file_path="c:/stefan/icu/datetimeevents.csv.gz",
    db_path="e:/mimic.db",
    table_name="datetimeevents"
)

load_csv_to_sqlite(
    file_path="c:/stefan/icu/d_items.csv.gz",
    db_path="e:/mimic.db",
    table_name="d_items"
)

"""



'\nload_csv_to_sqlite(\n    file_path="c:/stefan/icu/Ingredientevents.csv.gz",\n    db_path="e:/mimic.db",\n    table_name="Ingredientevents"\n)\n\nload_csv_to_sqlite(\n    file_path="c:/stefan/icu/inputevents.csv.gz",\n    db_path="e:/mimic.db",\n    table_name="inputevents"\n)\n\nload_csv_to_sqlite(\n    file_path="c:/stefan/icu/outputevents.csv.gz",\n    db_path="e:/mimic.db",\n    table_name="outputevents"\n)\n\nload_csv_to_sqlite(\n    file_path="c:/stefan/icu/procedureevents.csv.gz",\n    db_path="e:/mimic.db",\n    table_name="procedureevents"\n)\n\nload_csv_to_sqlite(\n    file_path="c:/stefan/icu/datetimeevents.csv.gz",\n    db_path="e:/mimic.db",\n    table_name="datetimeevents"\n)\n\nload_csv_to_sqlite(\n    file_path="c:/stefan/icu/d_items.csv.gz",\n    db_path="e:/mimic.db",\n    table_name="d_items"\n)\n\n'

In [9]:
# Belose are queryies we ran to see Query to figure out table information and which table we can take advantage of in order 
# to realize tables to include.


# Connect to (or create) a database file (SQL LITE database which will contain all input and output file. To utilize SQL.
# Name is mimic.db.
conn = sqlite3.connect("e:/mimic.db")

# Run a SQL query

query = "SELECT * FROM diagnoses_icd WHERE icd_code LIKE '806%';"
query1 = "SELECT distinct * FROM d_icd_diagnoses WHERE icd_code LIKE '806%';"
query2 = "SELECT * FROM d_icd_procedures WHERE icd_code LIKE '806%';"
query3 = "SELECT * FROM procedures_icd WHERE icd_code LIKE '806%';"
query4 = "select distinct(icd_code) from diagnoses_icd where icd_code LIKE '806%' order by 1;"

#SELECT * FROM diagnosis_icd where icd_code  limit 5;"
result_df4 = pd.read_sql_query(query4, conn)
result_df = pd.read_sql_query(query, conn)
result_df1 = pd.read_sql_query(query1, conn)
result_df2 = pd.read_sql_query(query2, conn) 
result_df3 = pd.read_sql_query(query3, conn)

# Display the query result
print(result_df4)
print(result_df)
print(result_df1)
print(result_df2)
print(result_df3)
conn.close()

   icd_code
0     80600
1     80601
2     80602
3     80603
4     80604
5     80605
6     80606
7     80607
8     80608
9     80609
10    80610
11    80613
12    80620
13    80621
14    80623
15    80624
16    80625
17    80626
18    80629
19    80639
20     8064
21     8065
22    80660
23    80662
24     8068
     subject_id   hadm_id  seq_num icd_code  icd_version
0      10035747  27083519        3    80610            9
1      10196368  24464472        1    80604            9
2      10229323  25045444       22     8064            9
3      10275408  20562387        1    80603            9
4      10275408  20562387        4    80608            9
..          ...       ...      ...      ...          ...
270    19953811  29850814        2    80624            9
271    19964212  26401796        1    80620            9
272    19970947  28546800        1    80600            9
273    19970947  28546800        2    80620            9
274    19970947  28546800        3    80605            9

[27

In [13]:
# Verification query of icd codes; ALL the Spinal realted icd_code (1136 Observations discovered).
# First driving table, to filter out the Spinal related injuries/issues.
query = """
SELECT
    icd_code,
    long_title
FROM
    d_icd_diagnoses
WHERE
    LOWER(long_title) LIKE '%spinal%'
    OR LOWER(long_title) LIKE '%spine%'
    OR LOWER(long_title) LIKE '%vertebral%'
ORDER BY
    icd_code;
"""
pd.set_option('display.max_rows', None)
result = execute_sql("e:/mimic.db", query)
if result is not None:
    print(result)



--------------- DataFrame Exploration Summary ---------------

Number of records (rows): 1136
Number of columns: 2

Column Names:
Index(['icd_code', 'long_title'], dtype='object')

First 5 rows of the DataFrame:
  icd_code                                         long_title
0     1340            Tuberculoma of spinal cord, unspecified
1     1341  Tuberculoma of spinal cord, bacteriological or...
2     1342  Tuberculoma of spinal cord, bacteriological or...
3     1343  Tuberculoma of spinal cord, tubercle bacilli f...
4     1344  Tuberculoma of spinal cord, tubercle bacilli n...

Last 5 rows of the DataFrame:
     icd_code                                         long_title
1131  W60XXXD  Contact with nonvenomous plant thorns and spin...
1132  W60XXXS  Contact with nonvenomous plant thorns and spin...
1133    Z1831                   Retained animal quills or spines
1134    Z4541  Encounter for adjustment and management of cer...
1135     Z982    Presence of cerebrospinal fluid drainage de

In [16]:
#Create a table(spinal_icd_list) in the SQL database that contains all the spinal number for linking; 
import sqlite3

# Connect to the local SQLite database
conn = sqlite3.connect('e:/mimic.db')
cur = conn.cursor()

# Create the new table using the SQL command
create_table_sql = """
CREATE TABLE spinal_icd_list AS
SELECT
    icd_code,
    long_title
FROM
    d_icd_diagnoses
WHERE
    LOWER(long_title) LIKE '%spinal%'
    OR LOWER(long_title) LIKE '%spine%'
    OR LOWER(long_title) LIKE '%vertebral%'
ORDER BY
    icd_code;
"""
cur.executescript(create_table_sql)
conn.commit()
conn.close()

In [14]:
# Query #1.
# Connect to (or create) a database file (Inital table with ICU patients that have spinal diagnosis, all 5,290 there could be
# multiple records as one patient can have multiple diagnosis.
# This is our driving table and contains all records related to indivdual who end up with spinal injures in the Intensive Care unit;
# The key is 5290 observation and that should not expand or decrease when adding more elements to the driving ICU patients table
# that have spinal chord related issues.

import sqlite3

# Connect to the local SQLite database
conn = sqlite3.connect('e:/mimic.db')
cur = conn.cursor()

# Create the new table using the SQL command
create_table_sql = """
CREATE TABLE ICU_SPINAL_PATIENT_5290 AS
SELECT
    i.subject_id,
    i.hadm_id,
    i.stay_id,
    i.first_careunit,
    i.last_careunit,
    i.intime,
    i.outtime,
    i.los,
    d.icd_code,
    s.long_title
FROM 
    icustays AS i
JOIN 
    diagnoses_icd AS d 
    ON i.hadm_id = d.hadm_id
JOIN 
    spinal_icd_list AS s 
    ON d.icd_code = s.icd_code
ORDER BY 
    i.subject_id, i.intime;
"""

cur.executescript(create_table_sql)
conn.commit()
conn.close()


In [15]:
# Query #2
# add to the ICU patient with additional columns such as gender, race, martial status,etc.
# Also if you see below the table and each characteristics and what is being added from the diffrent tables.
# Also as you can see we added the ICUSTAY table to the DIAGNOSTIC table and then to the SPINAL_ICD table then to the PATIENT table was 
# because there was not way to join the ICUSTAY table directly to the PATIENT table;
# See below, note that the last join was a left join so that not records were dropped due to the Pateint table possibly not having records 
# that matched up. 5,290 Observation was maintained.
import sqlite3

# Connect to the local SQLite database
conn = sqlite3.connect('e:/mimic.db')
cur = conn.cursor()

# Create the new table using the SQL command
create_table_sql = """
CREATE TABLE ICU_SPINAL_PATIENT_FEATURES AS
SELECT 
    i.subject_id,              -- ICU patient identifier
    i.hadm_id,                 -- Hospital admission ID
    i.stay_id,                 -- ICU stay ID
    i.intime,                  -- ICU admission time
    i.outtime,                 -- ICU discharge time
    i.los,                     -- ICU Length of Stay (target)
    d.icd_code,                -- ICD code from the diagnoses table
    s.long_title,              -- Description for the ICD code (spinal-related)
    p.gender,                  -- Demographic: Gender from the patients table
    p.anchor_age,              -- Age of the Client
    p.dod,                     -- Date of Death or Blank.
    a.admission_type,          -- Admission type (e.g., emergency, urgent)
    a.admission_location,      -- Location from which the patient was admitted
    a.insurance,               -- Insurance information (proxy for socioeconomic status)
    a.marital_status,          -- Marital status
    a.race                     -- Race
FROM icustays i
JOIN diagnoses_icd d 
  ON i.hadm_id = d.hadm_id
JOIN spinal_icd_list s 
  ON d.icd_code = s.icd_code
LEFT JOIN patient p 
  ON i.subject_id = p.subject_id
LEFT JOIN admissions a 
  ON i.hadm_id = a.hadm_id;
"""
cur.executescript(create_table_sql)
conn.commit()
conn.close()


In [9]:
# Understand one of the other tables we wanted to join. EDA, self learning/exploratory. 
# we are now using the function read_sql_query making code more readable, smaller, and consistent. 
# We are lookin to add more reading usally done by Nurses such as heartrate, etc.
# Note that there are close to 28 millions rows and multiples reading for each client.
# We will need to make sure that we transpose the data for the columns added and make sure
# we maintain the magical 5,290 Oberservation not increasing or decreasing the records.

conn = sqlite3.connect("e:/mimic.db")

# Run a SQL query

query = """
SELECT 
    ce.itemid,
    di.label,
    di.abbreviation,
    di.category,
    ce.valuenum,
    ce.charttime
FROM chartevents ce
JOIN d_items di ON ce.itemid = di.itemid
WHERE ce.itemid IN (220045, 227969, 220210, 220277)
ORDER BY ce.charttime;
"""

#SELECT * FROM diagnosis_icd where icd_code  limit 5;"
result_df = pd.read_sql_query(query, conn)

# Display the query result
#print(result_df)
explore_dataframe(result_df)

conn.close()

--------------- DataFrame Exploration Summary ---------------

Number of records (rows): 27949612
Number of columns: 6

Column Names:
Index(['itemid', 'label', 'abbreviation', 'category', 'valuenum', 'charttime'], dtype='object')

First 5 rows of the DataFrame:
   itemid                        label     abbreviation  \
0  220045                   Heart Rate               HR   
1  220210             Respiratory Rate               RR   
2  220277  O2 saturation pulseoxymetry             SpO2   
3  227969              Safety Measures  Safety Measures   
4  227969              Safety Measures  Safety Measures   

                    category  valuenum            charttime  
0        Routine Vital Signs      94.0  2110-01-11 12:42:00  
1                Respiratory      13.0  2110-01-11 12:42:00  
2                Respiratory     100.0  2110-01-11 12:42:00  
3  Restraint/Support Systems       NaN  2110-01-11 12:45:00  
4  Restraint/Support Systems       NaN  2110-01-11 12:45:00  

Last 5 row

In [16]:
# Query #3
# Columns: We use  for grouping.
# Aggregation: We compute the 
# average (), minimum (), and maximum () values from the  column for those records where , which is assumed to correspond to the heart rate measure.
# Grouping: The aggregation is done per ICU stay ().
# Using summary functions above to make sure we transpose the data and given there are many many records related to each client due to frequency
# of vitals taken for each patient we had to use the above summary functions to get some key attributes that might help us with the task.
# Description of each field is defined below by using comments.

"""
load_csv_to_sqlite(
    file_path="c:/stefan/icu/caregiver.csv.gz",
    db_path="e:/mimic.db",
    table_name="caregiver"
)
"""
import sqlite3

# Connect to the local SQLite database
conn = sqlite3.connect('e:/mimic.db')
cur = conn.cursor()

# Create the new table using the SQL command
create_table_sql = """
-- Create a new table that enriches ICU_SPINAL_PATIENT_FEATURES with aggregated vital sign measurements
CREATE TABLE ICU_SPINAL_PATIENT_FEATURES_VITALS AS
SELECT 
    -- Fields from the base table (ICU_SPINAL_PATIENT_FEATURES) with 1,136 rows
    f.subject_id,              -- Unique patient identifier
    f.hadm_id,                 -- Hospital admission identifier
    f.stay_id,                 -- ICU stay identifier
    f.intime,                  -- ICU admission time
    f.outtime,                 -- ICU discharge time
    f.los,                     -- ICU Length of Stay (target variable)
    f.icd_code,                -- ICD code from the initial diagnosis
    f.long_title,              -- Descriptive diagnosis (spinal-related)
    f.gender,                  -- Patient gender
    f.anchor_age,              -- Age of the Client
    f.dod,                     -- Date of Death or Blank.
    f.admission_type,          -- Type of admission (e.g., emergency, elective)
    f.admission_location,      -- Where the patient was admitted from
    f.insurance,               -- Insurance information (socioeconomic indicator)
    f.marital_status,          -- Marital status
    f.race,                    -- Patient race
    -- Aggregated vital signs from chartevents:
    ce.avg_hr,                 -- Average Heart Rate (itemid 220045)
    ce.min_hr,                 -- Minimum Heart Rate
    ce.max_hr,                 -- Maximum Heart Rate
    ce.avg_rr,                 -- Average Respiratory Rate (itemid 220210)
    ce.min_rr,                 -- Minimum Respiratory Rate
    ce.max_rr,                 -- Maximum Respiratory Rate
    ce.avg_SpO2,               -- Average Oxygen Saturation (SpO2) (itemid 220277)
    ce.min_SpO2,               -- Minimum Oxygen Saturation
    ce.max_SpO2                -- Maximum Oxygen Saturation
FROM ICU_SPINAL_PATIENT_FEATURES f
-- LEFT JOIN ensures that every row from the 1,136-row base table is preserved, even if no vital data is present
LEFT JOIN (
    -- Aggregates vital sign measurements from chartevents per ICU stay (stay_id)
    SELECT 
        stay_id,  -- Grouping key (matches ICU stay identifier)
        -- Aggregated measures for Heart Rate (itemid 220045)
        AVG(CASE WHEN itemid = 220045 THEN valuenum END) AS avg_hr,
        MIN(CASE WHEN itemid = 220045 THEN valuenum END) AS min_hr,
        MAX(CASE WHEN itemid = 220045 THEN valuenum END) AS max_hr,
        -- Aggregated measures for Respiratory Rate (itemid 220210)
        AVG(CASE WHEN itemid = 220210 THEN valuenum END) AS avg_rr,
        MIN(CASE WHEN itemid = 220210 THEN valuenum END) AS min_rr,
        MAX(CASE WHEN itemid = 220210 THEN valuenum END) AS max_rr,
        -- Aggregated measures for Oxygen Saturation (SpO2) (itemid 220277)
        AVG(CASE WHEN itemid = 220277 THEN valuenum END) AS avg_SpO2,
        MIN(CASE WHEN itemid = 220277 THEN valuenum END) AS min_SpO2,
        MAX(CASE WHEN itemid = 220277 THEN valuenum END) AS max_SpO2
    FROM chartevents
    WHERE itemid IN (220045, 220210, 220277)  -- Filter for the selected vital sign itemids
    GROUP BY stay_id
) ce
ON f.stay_id = ce.stay_id;  -- Join aggregated results by ICU stay identifier
"""
cur.executescript(create_table_sql)
conn.commit()
conn.close()


In [17]:
#Query #4 is very similar to query #3 as agian we used LABATORY table to add key attirbutes
#to the main ICUTABLE (summary again due to one to many realtionship as we want to maintain the magical 5290 records)
#Items such as white blood cell count.

import sqlite3

# Connect to the local SQLite database
conn = sqlite3.connect('e:/mimic.db')
cur = conn.cursor()

# Create the new table using the SQL command
create_table_sql = """
CREATE TABLE ICU_SPINAL_PATIENT_FEATURES_VITALS_EXT AS
SELECT 
    -- Basic Patient and ICU Admission Details:
    f.subject_id,              -- Unique patient identifier
    f.hadm_id,                 -- Hospital admission identifier
    f.stay_id,                 -- ICU stay identifier
    f.intime,                  -- ICU admission timestamp
    f.outtime,                 -- ICU discharge timestamp
    f.los,                     -- ICU length of stay (target variable)
    f.icd_code,                -- ICD code for spinal diagnosis
    f.long_title,              -- Detailed description of the spinal diagnosis
    f.gender,                  -- Patient gender
    f.anchor_age,              -- Patient age (from the 'anchor_age' field)
    f.dod,                     -- Date of death (if applicable)
    f.admission_type,          -- Type of admission (e.g., emergency, elective)
    f.admission_location,      -- Source/location of ICU admission
    f.insurance,               -- Insurance information (proxy for socioeconomic status)
    f.marital_status,          -- Marital status of the patient
    f.race,                    -- Patient race

    -- Vital Sign Aggregates from chartevents:
    ce.avg_hr,                 -- Average heart rate (itemid 220045) during ICU stay
    ce.min_hr,                 -- Minimum heart rate recorded during ICU stay
    ce.max_hr,                 -- Maximum heart rate recorded during ICU stay
    ce.avg_rr,                 -- Average respiratory rate (itemid 220210) during ICU stay
    ce.min_rr,                 -- Minimum respiratory rate recorded during ICU stay
    ce.max_rr,                 -- Maximum respiratory rate recorded during ICU stay
    ce.avg_SpO2,               -- Average oxygen saturation (SpO₂, itemid 220277) during ICU stay
    ce.min_SpO2,               -- Minimum oxygen saturation recorded during ICU stay
    ce.max_SpO2,               -- Maximum oxygen saturation recorded during ICU stay

    -- Additional Laboratory Values from labevents:
    lab.avg_creatinine,        -- Average serum creatinine (itemid 50912) during hospitalization
    lab.min_creatinine,        -- Minimum serum creatinine recorded
    lab.max_creatinine,        -- Maximum serum creatinine recorded
    labwbc.avg_wbc,            -- Average white blood cell (WBC) count (example: itemid 51300)
    labwbc.min_wbc,            -- Minimum WBC count recorded
    labwbc.max_wbc,            -- Maximum WBC count recorded

    -- Derived Timing Features from ICU Admission Time:
    strftime('%H', f.intime) AS admission_hour,    -- Hour of ICU admission (00 to 23)
    strftime('%w', f.intime) AS admission_dayofweek, -- Day-of-week (0 = Sunday, 6 = Saturday)
    CASE 
        WHEN strftime('%w', f.intime) IN ('0','6') THEN 1  -- Flag for weekend admission (1=Yes)
        ELSE 0 
    END AS weekend_admission,

    -- Comorbidity Indicator:
    dx.dx_count                -- Count of distinct diagnoses (proxy for comorbidity burden)

FROM ICU_SPINAL_PATIENT_FEATURES f

-- LEFT JOIN aggregated vital sign measurements from chartevents by ICU stay:
LEFT JOIN (
    SELECT 
        stay_id,
        -- Heart Rate (itemid 220045):
        AVG(CASE WHEN itemid = 220045 THEN valuenum END) AS avg_hr,
        MIN(CASE WHEN itemid = 220045 THEN valuenum END) AS min_hr,
        MAX(CASE WHEN itemid = 220045 THEN valuenum END) AS max_hr,
        -- Respiratory Rate (itemid 220210):
        AVG(CASE WHEN itemid = 220210 THEN valuenum END) AS avg_rr,
        MIN(CASE WHEN itemid = 220210 THEN valuenum END) AS min_rr,
        MAX(CASE WHEN itemid = 220210 THEN valuenum END) AS max_rr,
        -- Oxygen Saturation (SpO₂, itemid 220277):
        AVG(CASE WHEN itemid = 220277 THEN valuenum END) AS avg_SpO2,
        MIN(CASE WHEN itemid = 220277 THEN valuenum END) AS min_SpO2,
        MAX(CASE WHEN itemid = 220277 THEN valuenum END) AS max_SpO2
    FROM chartevents
    WHERE itemid IN (220045, 220210, 220277)  -- Include only the selected vital sign itemids
    GROUP BY stay_id
) ce
    ON f.stay_id = ce.stay_id  -- Join on ICU stay identifier to match each patient's ICU stay

-- LEFT JOIN aggregated serum creatinine values from labevents:
LEFT JOIN (
    SELECT 
        hadm_id,
        AVG(CASE WHEN itemid = 50912 THEN valuenum END) AS avg_creatinine,
        MIN(CASE WHEN itemid = 50912 THEN valuenum END) AS min_creatinine,
        MAX(CASE WHEN itemid = 50912 THEN valuenum END) AS max_creatinine
    FROM labevents
    WHERE itemid = 50912
    GROUP BY hadm_id
) lab
    ON f.hadm_id = lab.hadm_id  -- Join on hospital admission identifier

-- LEFT JOIN aggregated WBC count values from labevents:
LEFT JOIN (
    SELECT 
        hadm_id,
        AVG(CASE WHEN itemid = 51300 THEN valuenum END) AS avg_wbc,
        MIN(CASE WHEN itemid = 51300 THEN valuenum END) AS min_wbc,
        MAX(CASE WHEN itemid = 51300 THEN valuenum END) AS max_wbc
    FROM labevents
    WHERE itemid = 51300
    GROUP BY hadm_id
) labwbc
    ON f.hadm_id = labwbc.hadm_id  -- Join on hospital admission identifier

-- LEFT JOIN to derive a comorbidity indicator by counting distinct ICD codes:
LEFT JOIN (
    SELECT 
        hadm_id,
        COUNT(DISTINCT icd_code) AS dx_count
    FROM diagnoses_icd
    GROUP BY hadm_id
) dx 
    ON f.hadm_id = dx.hadm_id;  -- Join on hospital admission identifier
"""
cur.executescript(create_table_sql)
conn.commit()
conn.close()


In [2]:
# extract above final table to a .CSV file for sharing;

Data successfully exported to ICU_SPINAL_PATIENT_FEATURES_VITALS_EXT.csv


In [3]:
# Below is the final table exported out 5,290 records so our team can now do the Analysis on this table.
# For us to do analysis we need at lease 1K of records and we were lucky to have over 5K or records for analysis.

import sqlite3
import csv

# Path to your SQLite database on e drive
db_path = r"e:\mimic.db"

# Table name (and CSV file name)
table_name = "ICU_SPINAL_PATIENT_FEATURES_VITALS_EXT"

# Output CSV file path on e drive
csv_file = r"e:\ICU_SPINAL_PATIENT_FEATURES_VITALS_EXT.csv"

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Create a cursor object
cursor = conn.cursor()

# Query to fetch all data from the table
query = f"SELECT * FROM {table_name}"

try:
    # Execute the query
    cursor.execute(query)

    # Fetch all rows
    rows = cursor.fetchall()

    # Get column names
    column_names = [description[0] for description in cursor.description]

    # Write data to CSV file
    with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write column headers
        writer.writerow(column_names)

        # Write rows
        writer.writerows(rows)

    print(f"Data successfully exported to {csv_file}")
except sqlite3.Error as e:
    print(f"An error occurred: {e}")
finally:
    # Close the connection
    conn.close()

Data successfully exported to e:\ICU_SPINAL_PATIENT_FEATURES_VITALS_EXT.csv
